## Reduced Rank Regression

In [70]:
import os
import pickle as pkl
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from utils.helpers import *

# autoreload
%reload_ext autoreload
%autoreload 2

In [3]:
# Load the data
current_dir = os.getcwd()
data_path = os.path.join(current_dir, 'data')
data = load_data(data_path, 'ssp585_time_series.pkl')

In [7]:
filtered_data = filter_data(data)

In [12]:
# Create a NaN mask
nan_mask = create_nan_mask(filtered_data)

# Apply the NaN mask
masked_data = remove_nans(filtered_data, nan_mask)

100%|██████████| 34/34 [00:02<00:00, 13.89it/s]


In [20]:
reshaped_data = reshape_data(masked_data)

In [26]:
example_model = 'CanESM5-1'
example_model_data = reshaped_data[example_model]
print(f"Randomly selected model: {example_model}, which has {len(example_model_data)} runs.")

Randomly selected model: CanESM5-1, which has 72 runs.


In [28]:
first_example_model_run = example_model_data[list(example_model_data.keys())[0]]
first_example_model_run.shape # Shape: (Time x (Latitude x Longitude))

(165, 10368)

In [ ]:
# Keep only the first 5 models as an example
example_models = list(reshaped_data.keys())[:2]
example_models_data = {model: reshaped_data[model] for model in example_models}

In [34]:
# Now Normalizing the reshaped data
normalized_data, scaler = normalize_data(example_models_data)

c:\ProgramData\anaconda3\envs\Project\lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\ProgramData\anaconda3\envs\Project\lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\ProgramData\anaconda3\envs\Project\lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
100%|██████████| 5/5 [00:00<00:00,  6.54it/s]


In [ ]:
# Shape of the 1st run of the 1st model
normalized_data[example_models[0]][list(normalized_data[example_models[0]].keys())[0]].shape # Shape (Time x (Latitude x Longitude))

(165, 10368)

In [55]:
data_forced_responses = add_forced_response(normalized_data)

### The data is now normalized with the forced response being calculated (on the normalized data)

In [63]:
X_all, y_all = pool_data(data_forced_responses)

In [69]:
X_all.shape, y_all.shape

((10725, 10368), (10725, 10368))

In [ ]:
# Perform the Reduced Rank Regression
B_rrr = reduced_rank_regression(X_all, y_all, rank=2)

MemoryError: Unable to allocate 848. MiB for an array with shape (10725, 10368) and data type float64